In [ ]:
!wget https://www.dropbox.com/scl/fo/3lhxmvlbb4mq7ilqud9dw/AAnsdiVGBUJC2b5xrOs9Lb8?rlkey=emxvs4sno8migvs0tjljufwel&st=spwxnfjn&dl=1
!git clone https://github.com/a3d2ddk/Prenith-and-Drew-Camera-Calibration-Assingment

In [10]:
!pip install numpy
!pip install opencv-python
!pip install matplotlib
!pip install gradio
!pip install pillow
!pip install pytransform3d


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [34]:
import cv2 as cv
import numpy as np
import gradio as gr
import matplotlib.pyplot as plt
import pytransform3d.camera as pc
import pytransform3d.transformations as pt
import json
import glob
import os


from PIL import Image

In [35]:
class Img:
    """Camera math: poses, transforms, calibration"""
    
    @staticmethod
    # Takes an array of openCV images
    def find_chessboard_corners(images, board_size=[9, 6]):
        """Returns list of points in real space and on the image plane"""
        # termination criteria
        criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

        # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
        objp = np.zeros((board_size[1]*board_size[0],3), np.float32)
        objp[:,:2] = np.mgrid[0:board_size[0],0:board_size[1]].T.reshape(-1,2)

        # Arrays to store object points and image points from all the images.
        objpoints = [] # 3d point in real world space
        imgpoints = [] # 2d points in image plane.

        for img in images:
            # Find the chess board corners
            ret, corners = cv.findChessboardCorners(img, (board_size[0],board_size[1]), None)

            # If found, add object points, image points (after refining them)
            if ret == True:
                objpoints.append(objp)

                corners2 = cv.cornerSubPix(img,corners, (11,11), (-1,-1), criteria)
                imgpoints.append(corners2)

        return objpoints, imgpoints
    
    @staticmethod
    # Takes openCV Image
    def undistort_image(image, lam, dist):
        h, w = image.shape[:2]
        newcameramtx, roi = cv.getOptimalNewCameraMatrix(lam, dist, (w,h), 1, (w,h))

        # undistort
        dst = cv.undistort(image, lam, dist, None, newcameramtx)

        # crop the image
        x, y, w, h = roi
        dst = dst[y:y+h, x:x+w]

        return dst

class Cam:
    """Image utilities: loading, preprocessing, format conversion"""
        
    @staticmethod
    def get_coord_frame(rvec, tau, lam, dist) -> np.ndarray:
        W = 2 * np.array([
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
            ], dtype=np.float64)
        
        image_axes, _ = cv.projectPoints(W, rvec, tau, lam, dist)
        image_axes = image_axes.squeeze().T

        return image_axes

    @staticmethod
    def get_camera_pos (image, omega, tau) -> np.ndarray:
        pose = np.block([omega.T, -omega.T @ tau])
        pose = np.vstack([pose, [0, 0, 0, 1]])

        return pose

class IO:
    """File and device I/O utilities"""
    
    @staticmethod
    def store_images(dir_path):
        # Reads all images and saves them to content/images/
        for file in dir_path:
            image = os.path.basename(file)
            if (image.lower().endswith('.jpeg')) and (os.path.exists(file)):
                # Images read as grayscale
                img = cv.imread(file, cv.IMREAD_GRAYSCALE)

                cv.imwrite('./content/images/' + image, img)

            else:
                print(f'File: {image} must be a .jpeg file.')

        return
            
    @staticmethod
    def get_images():
        # Reads all image files in content/images
        img_list = glob.glob('./content/images/*.jpeg')
        images = []
        for img in img_list:
            img = cv.imread(img, cv.IMREAD_GRAYSCALE)
            images.append(img)
        return images

    @staticmethod
    def remove_images(files):
        # Deletes specified images in
        for frame in files:
            try:
                os.remove('./content/images/' + frame)
            except:
                raise ValueError(f'Could not delete File: {frame}')
        return

In [ ]:
# Main calibration workflow
class CalibrationWorkflow:
    """Main calibration workflow using static helper classes"""
    
    def __init__(self, board_size=(9, 6), square_size=25.0, output_dir="/content/images"):
        self.board_size = board_size
        self.square_size = square_size
        self.output_dir = output_dir
        self.obj_points = []
        self.img_points = []
        self.processed_images = []
        self.calibration_results = None
    
    def process_images(self, files):
        """Process uploaded images and detect chessboard corners"""
        imgs = IO.get_images()
        if not files and not imgs:
            return "No files uploaded!"

        IO.store_images(files)
        self.imgs = IO.get_images()

        self.opoints, self.ipoints = Img.find_chessboard_corners(imgs)

        ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(
        self.opoints, self.ipoints, 
        self.imgs[0].shape[::-1], None, None
        )
        
        # Store all calibration results including individual poses
        self.calibration_results = {
            'mtx': mtx.tolist(),
            'dist': dist.tolist(),
            'rvecs': [rvec.tolist() for rvec in rvecs],  # All individual rvecs
            'tvecs': [tvec.tolist() for tvec in tvecs],  # All individual tvecs
            'ret': ret
        }
        
        # Save to JSON file
        import json
        with open('calibration_results.json', 'w') as f:
            json.dump(self.calibration_results, f, indent=2)
        
        self.processed_images = [f"image_{i}.jpg" for i in range(len(self.imgs))]

        return "Calibration Complete"
    
    def get_camera_pose_plot(self):
        """Get camera pose visualization"""

        with open('calibration_results.json', 'r') as f:
            self.calibration_results = json.load(f)
            
        lam = np.array(self.calibration_results['mtx'])

        # Prepare the figure for plotting
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        nRows, nCols = self.imgs[0].shape

        sensor_size = np.array([nCols, nRows])

        # Loop through each camera pose
        img = self.imgs[0]
        rvec = np.array(self.calibration_results['rvecs'][0])
        tvec = np.array(self.calibration_results['tvecs'][0])
        
        # Create the camera pose matrix
        R, _ = cv.Rodrigues(rvec)
        Rt = np.block([R.T, -R.T @ tvec.reshape(3, 1)])
        Rt = np.vstack([Rt, [0, 0, 0, 1]])

        ax = pt.plot_transform(A2B=Rt, s=2)

        # Plot the camera
        pc.plot_camera(ax, cam2world=Rt, M=lam, sensor_size=sensor_size, virtual_image_distance=0.8)

        # Set limits and view angle
        ax.set_xlim(-10, 10)
        ax.set_ylim(-10, 10)
        ax.set_zlim(-30, 30)
        ax.view_init(30, 70)
        plt.grid()

        # Save the plot to a file and return the file path
        plt.savefig("content/camera_pose_plot.png")
        plt.close(fig)  # Close the figure to avoid display issues
        return "content/camera_pose_plot.png"  # Return the path to the saved image
    
    def get_sample_images_with_axes(self, num_samples=8):
        """Get sample images with coordinate axes drawn"""
        images = []
        
        with open('calibration_results.json', 'r') as f:
            self.calibration_results = json.load(f)

        lam = np.array(self.calibration_results['mtx'])
        dist = np.array(self.calibration_results['dist'])

        for i in range(5):
            img = self.imgs[i]

            rvecs = np.array(self.calibration_results['rvecs'][i])
            tvecs = np.array(self.calibration_results['tvecs'][i])

            
            omega = cv.Rodrigues(rvecs)
            omega = omega[0]
            
            W = 2 * np.array([
                [0, 1, 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]
            ], dtype=np.float64).T

            image_axes, jac = cv.projectPoints(W, rvecs, tvecs, lam, dist)

            image_axes = image_axes.squeeze().T

            x0, y0 = image_axes[:,0].astype(int)
            cv.circle(img, (x0, y0), 9, (0, 0, 0), -1)

            x1, y1 = image_axes[:,1].astype(int)
            img = cv.arrowedLine(img, (x0, y0), (x1, y1), (255, 0, 0), 5)

            x2, y2 = image_axes[:,2].astype(int)
            img = cv.arrowedLine(img, (x0, y0), (x2, y2), (0, 255, 0), 5)

            x3, y3 = image_axes[:,3].astype(int)
            img = cv.arrowedLine(img, (x0, y0), (x3, y3), (0, 0, 255), 5)

            pil = Image.fromarray(img)

            images.append(pil)

        return images[0], images[1], images[2], images[3], images[4]
    
    def get_undistortion_preview(self):
        """Get undistortion before/after comparison"""
        test_img = self.imgs[18]
        ret, lam, dist, rvecs, tau = cv.calibrateCamera(self.opoints, self.ipoints, test_img.shape[::-1], None, None)

        new_img = Img.undistort_image(test_img, lam, dist)

        original_pil = Image.fromarray(test_img)
        undistorted_pil = Image.fromarray(new_img)

        return undistorted_pil, undistorted_pil

In [ ]:
# Create workflow instance
workflow = CalibrationWorkflow()

# Gradio interface
with gr.Blocks(title="Camera Calibration") as demo:
    gr.Markdown("#Camera Calibration Interface")
    gr.Markdown("Upload chessboard images and run calibration with visualization")
    
    with gr.Row():
        files = gr.File(file_count="multiple", file_types=[".jpeg"])
        process_btn = gr.Button("Process Images")
    
    process_status = gr.Textbox(label="Processing Status")
    
    with gr.Tabs():
        with gr.Tab("Camera Poses"):
            pose_btn = gr.Button("Show Camera Poses")
            pose_plot = gr.Image()
        
        with gr.Tab("Sample Images"):
            sample_btn = gr.Button("Show Images with Axes")
            with gr.Row():
                img1 = gr.Image(label="Sample 1")
                img2 = gr.Image(label="Sample 2")
                img3 = gr.Image(label="Sample 3")
                img4 = gr.Image(label="Sample 4")
                img5 = gr.Image(label="Sample 5")
        
        with gr.Tab("Undistortion"):
            undist_btn = gr.Button("Show Undistortion Preview")
            with gr.Row():
                original = gr.Image(label="Original")
                undistorted = gr.Image(label="Undistorted")
    
    # Event handlers
    process_btn.click(
        workflow.process_images, 
        inputs=[files], 
        outputs=[process_status]
    )
    
    pose_btn.click(
        workflow.get_camera_pose_plot, 
        outputs=[pose_plot]
    )
    
    sample_btn.click(
        workflow.get_sample_images_with_axes, 
        outputs=[img1, img2, img3, img4, img5]
    )
    
    undist_btn.click(
        workflow.get_undistortion_preview, 
        outputs=[original, undistorted]
    )

# Launch interface
app = demo.launch(share=True, debug=False)

* Running on local URL:  http://127.0.0.1:7871
* Running on public URL: https://fba04f981b3951a969.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[[-0.26391652]
 [-0.31945995]
 [ 2.82152694]]
[[ 3.95245979]
 [ 1.73052977]
 [21.18592906]]
